In [1]:
import pandas as pd

In [2]:
TASK = "forming"

In [3]:
field_name_maps = {
    "rolling": {
        "all": "All Fields Normalized Avg (-)",
        "deformation": "Deformation (mm)",
        "nodes_LE": "Logarithmic Strain (-)",
        "nodes_PEEQ": "Equivalent Plastic Strain (-)",
        "nodes_mises_stress": "Mises Stress (MPa)",
        "nodes_stresses": "Stress (MPa)"
    },
    "forming": {
        "all": "All Fields Normalized Avg (-)",
        "deformation": "Deformation (mm)",
        "nodes_LE": "Logarithmic Strain (-)",
        "nodes_PEEQ": "Equivalent Plastic Strain (-)",
        "nodes_mises_stress": "Mises Stress (MPa)",
        "nodes_stresses": "Stress (MPa)"
    },
    "motor": {
        "all": "All Fields Normalized Avg (-)",
        "deformation": "Deformation (m)",
        "logarithmic_strain": "Logarithmic Strain (-)",
        "principal_strain": "Principal Strain (-)",
        "stress": "Stress (MPa)",
        "stress_cauchy": "Cauchy Stress (MPa)",
        "stress_mises": "Mises Stress (MPa)",
        "stress_principal": "Principal Stress (MPa)",
        "total_strain": "Total Strain (-)",
    }
}

In [4]:
# df = pd.read_pickle(f"tables/results_{TASK}_backup.pkl")
df = pd.read_pickle(f"model_selection_results/results_{TASK}.pkl")
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156 entries, 0 to 155
Data columns (total 16 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   model_name                           156 non-null    object 
 1   da_algorithm_name                    156 non-null    object 
 2   model_selection_algorithm_name       156 non-null    object 
 3   seed                                 156 non-null    int64  
 4   test_loss_source                     156 non-null    float64
 5   test_loss_target                     156 non-null    float64
 6   test_loss_source_deformation         156 non-null    float64
 7   test_loss_target_deformation         156 non-null    float64
 8   test_loss_source_nodes_LE            156 non-null    float64
 9   test_loss_target_nodes_LE            156 non-null    float64
 10  test_loss_source_nodes_PEEQ          156 non-null    float64
 11  test_loss_target_nodes_PEEQ     

In [5]:
group_cols = ["model_name", "da_algorithm_name", "model_selection_algorithm_name"]

# pick out all your test-loss columns
loss_cols = [c for c in df.columns if c.startswith("test_loss_")]

# build agg dict: each loss_col → [mean, std]
agg_dict = {col: ["mean", "std"] for col in loss_cols}

# do the groupby-agg
agg_df = (
    df
    .groupby(group_cols)[loss_cols]
    .agg(agg_dict)
    .reset_index()
)

# flatten the MultiIndex columns
agg_df.columns = [
    f"{col}_{stat}" if stat else col
    for col, stat in agg_df.columns.to_flat_index()
]

print(agg_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 27 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   model_name                                39 non-null     object 
 1   da_algorithm_name                         39 non-null     object 
 2   model_selection_algorithm_name            39 non-null     object 
 3   test_loss_source_mean                     39 non-null     float64
 4   test_loss_source_std                      39 non-null     float64
 5   test_loss_target_mean                     39 non-null     float64
 6   test_loss_target_std                      39 non-null     float64
 7   test_loss_source_deformation_mean         39 non-null     float64
 8   test_loss_source_deformation_std          39 non-null     float64
 9   test_loss_target_deformation_mean         39 non-null     float64
 10  test_loss_target_deformation_std        

In [6]:
if TASK == "motor":
    for col in list(agg_df.columns):
        if "stress" in col:
            agg_df[col] = agg_df[col] /1e6

In [7]:
agg_df

,model_name,da_algorithm_name,model_selection_algorithm_name,test_loss_source_mean,test_loss_source_std,test_loss_target_mean,test_loss_target_std,test_loss_source_deformation_mean,test_loss_source_deformation_std,test_loss_target_deformation_mean,...,test_loss_target_nodes_PEEQ_mean,test_loss_target_nodes_PEEQ_std,test_loss_source_nodes_mises_stress_mean,test_loss_source_nodes_mises_stress_std,test_loss_target_nodes_mises_stress_mean,test_loss_target_nodes_mises_stress_std,test_loss_source_nodes_stresses_mean,test_loss_source_nodes_stresses_std,test_loss_target_nodes_stresses_mean,test_loss_target_nodes_stresses_std
0,GraphSAGE,-,-,0.070399,0.001862,0.376070,0.027757,1.410845,0.070177,1.938868,...,0.002722,0.000262,11.022072,0.323813,46.096513,4.910559,5.547897,0.197653,31.225296,1.553755
1,GraphSAGE,DANN,DEV,0.056467,0.003685,2.972798,4.353413,1.346591,0.044721,16.199442,...,0.027416,0.041561,10.597062,0.563725,406.576338,403.134538,5.334488,0.298952,177.376366,187.163974
2,GraphSAGE,DANN,IWV,0.056870,0.002812,0.329011,0.027009,1.405812,0.070954,2.095070,...,0.002695,0.000112,10.758415,0.277389,52.400813,7.908412,5.386955,0.133796,34.644105,4.403947
3,GraphSAGE,DANN,SB,0.055209,0.001795,1.139378,0.411187,1.403520,0.034707,7.809819,...,0.009206,0.004519,10.732401,0.405508,186.097961,37.057425,5.372326,0.167518,94.369928,16.942915
4,GraphSAGE,DANN,TB,0.056941,0.002941,0.322553,0.024885,1.415970,0.055409,2.020940,...,0.002646,0.000043,10.728022,0.217607,49.234445,5.605965,5.405231,0.167316,33.375273,4.785820
5,GraphSAGE,cmd,DEV,0.055327,0.002318,0.857019,0.474972,1.354761,0.057981,6.408831,...,0.006450,0.002898,10.589932,0.343409,145.233232,79.729507,5.287418,0.139502,87.123444,42.179088
6,GraphSAGE,cmd,IWV,0.055497,0.001181,0.406584,0.124290,1.326189,0.030681,2.454858,...,0.003577,0.001121,10.729610,0.065099,61.684642,20.292530,5.354454,0.114989,37.003723,7.363669
7,GraphSAGE,cmd,SB,0.054865,0.000808,0.569025,0.305816,1.432890,0.024029,4.708164,...,0.004969,0.002730,10.549628,0.162508,99.068965,58.189582,5.298693,0.070542,55.133526,35.743714
8,GraphSAGE,cmd,TB,0.057135,0.000952,0.289170,0.036123,1.344732,0.059042,2.027628,...,0.002427,0.000285,10.827523,0.169226,43.746307,5.836385,5.436507,0.109965,29.605521,3.477960
9,GraphSAGE,deep_coral,DEV,0.053702,0.001319,0.411344,0.103072,1.346939,0.047622,3.347486,...,0.003296,0.000644,10.355447,0.455309,65.860613,28.276884,5.206255,0.189791,43.062116,14.129428


In [8]:
import pandas as pd
import re

def df_to_latex_table_with_highlights(
    df: pd.DataFrame,
    caption: str,
    label: str,
    float_fmt: str = "{:.4f}",
    threshold_factor: float = 100.0,
    da_name_map: dict = None,
    field_name_map: dict = None,
    exclude_selection: str = "TB"
) -> str:
    """
    Generate LaTeX table with:
      - Translated DA and field names,
      - mean±std cells or ★ for outliers,
      - multirow by model, cmidrule separation,
      - bold global lowest 'all' TARGET mean, underline per-model lowest,
        excluding rows with model_selection == exclude_selection.
    """
    # Defaults
    if da_name_map is None:
        da_name_map = {"deep_coral": "Deep Coral", "cmd": "CMD", "DANN": "DANN"}
    if field_name_map is None:
        field_name_map = {
            "all": "All Fields Normalized Avg (-)",
            "deformation": "Deformation (mm)",
            "nodes_LE": "Logarithmic Strain (-)",
            "nodes_PEEQ": "Equivalent Plastic Strain (-)",
            "nodes_mises_stress": "Mises Stress (Pa)",
            "nodes_stresses": "Stress (Pa)"
        }

    # Detect fields
    field_map = {}
    pat = re.compile(r"^test_loss_(source|target)(?:_(.+?))?_(mean|std)$")
    for c in df.columns:
        m = pat.match(c)
        if m:
            dom, fld, stat = m.groups()
            base = fld or "all"
            field_map.setdefault(base, {}).setdefault(dom.upper(), {})[stat] = c

    # Compute medians
    mean_cols = [c for c in df.columns if c.endswith("_mean")]
    med = df[mean_cols].median()

    # Highlight thresholds
    all_tgt_mean = field_map['all']['TARGET']['mean']
    global_min = df[df['model_selection_algorithm_name'] != exclude_selection][all_tgt_mean].min()
    per_model_min = df[df['model_selection_algorithm_name'] != exclude_selection]\
        .groupby('model_name')[all_tgt_mean].min().to_dict()

    # Build header
    fields = sorted(field_map.keys(), key=lambda x: (x!='all', x))
    nf = len(fields)
    total_cols = 3 + 2*nf
    lines = [
        r"\begin{table}[h]",
        r"  \centering",
        f"  \\caption{{{caption}}}",
        f"  \\label{{{label}}}",
        r"  \resizebox{\textwidth}{!}{%",
        "  \\begin{tabular}{" + "lll" + "c"*(2*nf) + "}",
        "    \\toprule"
    ]
    # Header rows
    hdr1 = (
        r"    \multirow{2}{*}{Model}"
        r" & \multirow{2}{*}{\makecell{DA\\Algorithm}}"
        r" & \multirow{2}{*}{\makecell{Model\\Selection}}"
    )
    for f in fields:
        hdr1 += f" & \\multicolumn{{2}}{{c}}{{{field_name_map.get(f,f)}}}"
    hdr1 += r" \\"
    lines.append(hdr1)
    cm = "    "
    for i in range(nf):
        cm += f"\\cmidrule(lr){{{4+2*i}-{5+2*i}}} "
    lines.append(cm.strip())
    hdr2 = "      &   &  & " + " & ".join(["SRC & TGT"]*nf) + r" \\"
    lines.extend([hdr2, "    \\midrule"])

    # Body
    for mi, model in enumerate(df['model_name'].unique()):
        sub = df[df['model_name']==model].sort_values(['da_algorithm_name','model_selection_algorithm_name'])
        n = len(sub)
        prev_da = None
        for idx, row in sub.iterrows():
            # block sep
            if prev_da is not None and row['da_algorithm_name']!=prev_da:
                lines.append(f"    \\cmidrule(lr){{2-{total_cols}}}")
            da = da_name_map.get(row['da_algorithm_name'], row['da_algorithm_name'])
            sel = row['model_selection_algorithm_name']
            is_global = (sel!=exclude_selection and row[all_tgt_mean]==global_min)
            is_model = (sel!=exclude_selection and row[all_tgt_mean]==per_model_min.get(model))
            # start row
            if idx==sub.index[0]:
                line = f"    \\multirow{{{n}}}{{*}}{{{model}}} & "
            else:
                line = "    & "
            # underline DA and sel if per-model best
            da_cell = f"\\underline{{{da}}}" if is_model else da
            sel_cell = f"\\underline{{{sel}}}" if is_model else sel
            line += f"{da_cell} & {sel_cell}"
            # fields
            for f in fields:
                for dom in ("SOURCE","TARGET"):
                    mc = field_map[f][dom]['mean']
                    sc = field_map[f][dom]['std']
                    mv, sv = row[mc], row[sc]
                    if mv > threshold_factor * med[mc]:
                        cell = r"$\star$"
                    else:
                        txt = f"{float_fmt.format(mv)}(\\pm{float_fmt.format(sv)})"
                        if f=='all' and dom=='TARGET':
                            if is_global:
                                cell = f"$\\underline{{\\mathbf{{{txt}}}}}$"
                            elif is_model:
                                cell = f"$\\underline{{{txt}}}$"
                            else:
                                cell = f"${txt}$"
                        else:
                            cell = f"${txt}$"
                    line += f" & {cell}"
            line += r" \\"
            lines.append(line)
            prev_da = row['da_algorithm_name']
        lines.append("    \\midrule" if mi < df['model_name'].nunique()-1 else "    \\bottomrule")

    lines.extend([r"  \end{tabular}",r"  }",r"\end{table}"])
    return "\n".join(lines)

In [9]:
table = (df_to_latex_table_with_highlights(
    agg_df,
    caption="Performance across different fields.",
    label="tab:fixed",
    da_name_map={"deep_coral":"Deep Coral","cmd":"CMD","DANN":"DANN"},
    field_name_map=field_name_maps[TASK],
    threshold_factor=10
))

print(table)

\begin{table}[h]
  \centering
  \caption{Performance across different fields.}
  \label{tab:fixed}
  \resizebox{\textwidth}{!}{%
  \begin{tabular}{lllcccccccccccc}
    \toprule
    \multirow{2}{*}{Model} & \multirow{2}{*}{\makecell{DA\\Algorithm}} & \multirow{2}{*}{\makecell{Model\\Selection}} & \multicolumn{2}{c}{All Fields Normalized Avg (-)} & \multicolumn{2}{c}{Deformation (mm)} & \multicolumn{2}{c}{Logarithmic Strain (-)} & \multicolumn{2}{c}{Equivalent Plastic Strain (-)} & \multicolumn{2}{c}{Mises Stress (MPa)} & \multicolumn{2}{c}{Stress (MPa)} \\
\cmidrule(lr){4-5} \cmidrule(lr){6-7} \cmidrule(lr){8-9} \cmidrule(lr){10-11} \cmidrule(lr){12-13} \cmidrule(lr){14-15}
      &   &  & SRC & TGT & SRC & TGT & SRC & TGT & SRC & TGT & SRC & TGT & SRC & TGT \\
    \midrule
    \multirow{13}{*}{GraphSAGE} & - & - & $0.0704(\pm0.0019)$ & $0.3761(\pm0.0278)$ & $1.4108(\pm0.0702)$ & $1.9389(\pm0.5299)$ & $0.0002(\pm0.0000)$ & $0.0016(\pm0.0001)$ & $0.0004(\pm0.0000)$ & $0.0027(\pm0.0003)$ &